# AutoGluon Time Series - Forecasting Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/master/docs/tutorials/timeseries/forecasting-quick-start.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/master/docs/tutorials/timeseries/forecasting-quick-start.ipynb)


Via a simple `fit()` call, AutoGluon can train and tune

- simple forecasting models (e.g., ARIMA, ETS, Theta),
- powerful deep learning models (e.g., DeepAR, Temporal Fusion Transformer),
- tree-based models (e.g., LightGBM),
- an ensemble that combines predictions of other models

to produce multi-step ahead _probabilistic_ forecasts for univariate time series data.

This tutorial demonstrates how to quickly start using AutoGluon to generate hourly forecasts for the [M4 forecasting competition](https://www.sciencedirect.com/science/article/pii/S0169207019301128) dataset.

## Loading time series data as a `TimeSeriesDataFrame`

First, we import some required modules

In [2]:
# We use uv for faster installation
!pip install uv
!uv pip install -q autogluon.timeseries --system
!uv pip uninstall -q torchaudio torchvision torchtext --system # fix incompatible package versions on Colab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 83.1 MB/s eta 0:00:00


In [20]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor


To use `autogluon.timeseries`, we will only need the following two classes:

- `TimeSeriesDataFrame` stores a dataset consisting of multiple time series.
- `TimeSeriesPredictor` takes care of fitting, tuning and selecting the best forecasting models, as well as generating new forecasts.

We load a subset of the M4 hourly dataset as a `pandas.DataFrame`

In [25]:
# df = pd.read_csv("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/train.csv")
df = pd.read_csv("/content/Forecast_File.csv")
df["Posting Date"] = pd.to_datetime(df["Posting Date"])
df.head()


,Posting Date,Amount,index_num
0,2022-01-05,19420.72,0
1,2022-02-23,19420.72,0
2,2022-03-22,19418.40,0
3,2022-05-23,29127.60,0
4,2022-06-08,19418.40,0


AutoGluon expects time series data in [long format](https://doc.dataiku.com/dss/latest/time-series/data-formatting.html#long-format).
Each row of the dataframe contains a single observation (timestep) of a single time series represented by

- unique ID of the time series (`"item_id"`) as int or str
- timestamp of the observation (`"timestamp"`) as a `pandas.Timestamp` or compatible format
- numeric value of the time series (`"target"`)

The raw dataset should always follow this format with at least three columns for unique ID, timestamp, and target value, but the names of these columns can be arbitrary.
It is important, however, that we provide the names of the columns when constructing a `TimeSeriesDataFrame` that is used by AutoGluon.
AutoGluon will raise an exception if the data doesn't match the expected format.

In [ ]:
# Create a complete date range from Jan 1, 2022 to March 31, 2025
date_range = pd.date_range(start="2022-01-01", end="2025-03-31", freq="D")

# Create a DataFrame with all dates and unique index_num values
# Assuming index_num should be preserved or incremented; here, we'll use the original index_num
full_df = pd.DataFrame({
    "Posting Date": date_range.repeat(df["index_num"].nunique())  # Repeat dates for each unique index_num
})

# Merge with original data to map Amount values
# Since index_num is all 0, we'll forward fill Amount based on the nearest available date
full_df = full_df.merge(df, on="Posting Date", how="left")
full_df["Amount"] = full_df["Amount"].fillna(method="ffill")  # Forward fill Amount
full_df["index_num"] = full_df["index_num"].fillna(0)  # Fill index_num with 0

# Reset index if needed
full_df = full_df.sort_values("Posting Date").reset_index(drop=True)

# Display the first few rows
print(full_df.head())

In [22]:
ts_data = TimeSeriesDataFrame.from_data_frame(
    full_df,
    id_column="index_num",
    timestamp_column="Posting Date"
)
ts_data.head()


Amount
item_id timestamp           
0       2022-01-05  19420.72
        2022-02-23  19420.72
        2022-03-22  19418.40
        2022-05-23  29127.60
        2022-06-08  19418.40

We refer to each individual time series stored in a `TimeSeriesDataFrame` as an _item_.
For example, items might correspond to different products in demand forecasting, or to different stocks in financial datasets.
This setting is also referred to as a _panel_ of time series.
Note that this is *not* the same as multivariate forecasting — AutoGluon generates forecasts for each time series individually, without modeling interactions between different items (time series).

`TimeSeriesDataFrame` inherits from [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), so all attributes and methods of `pandas.DataFrame` are available in a `TimeSeriesDataFrame`.
It also provides other utility functions, such as loaders for different data formats (see [TimeSeriesDataFrame](../../api/autogluon.timeseries.TimeSeriesDataFrame) for details).

## Training time series models with `TimeSeriesPredictor.fit`
To forecast future values of the time series, we need to create a `TimeSeriesPredictor` object.

Models in `autogluon.timeseries` forecast time series _multiple steps_ into the future.
We choose the number of these steps — the _prediction length_ (also known as the _forecast horizon_) —  depending on our task.
For example, our dataset contains time series measured at hourly _frequency_, so we set `prediction_length = 48` to train models that forecast up to 48 hours into the future.

We instruct AutoGluon to save trained models in the folder `./autogluon-m4-hourly`.
We also specify that AutoGluon should rank models according to [mean absolute scaled error (MASE)](https://en.wikipedia.org/wiki/Mean_absolute_scaled_error), and that data that we want to forecast is stored in the column `"target"` of the `TimeSeriesDataFrame`.

In [23]:
# Set forecasting length
prediction_length = 31

# Split into train/test
train_data = ts_data.slice_by_timestep(None, -prediction_length)
test_data = ts_data.slice_by_timestep(-prediction_length, None)

# ----------------------------
# STEP: Train AutoGluon predictor
# ----------------------------

predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    target="Amount",
    eval_metric="WAPE",  # or "MAPE", "MASE", etc.
    path="AutogluonModels/",
    freq="D"
)

# High-quality preset: takes longer but gives best accuracy
predictor.fit(
    train_data,
    presets="best_quality",  # or "medium_quality" for faster results
    time_limit=600  # seconds; change as needed
)

# ----------------------------
# STEP: Forecast
# ----------------------------

predictions = predictor.predict(test_data)


Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to '/content/AutogluonModels'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       10.76 GB / 12.67 GB (84.9%)
Disk Space Avail:   66.30 GB / 107.72 GB (61.6%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WAPE,
 'freq': 'D',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 31,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'Amount',
 'time_limit': 600,
 'verbosity': 2}

train_data with frequency 'IRREG' has been resampled to freq

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/821M [00:00<?, ?B/s]

	Time limit exceeded... Skipping ChronosZeroShot[bolt_base].
Training timeseries model ChronosFineTuned[bolt_small]. Training for up to 37.6s of the 225.4s of remaining time.
	Skipping covariate_regressor since the dataset contains no known_covariates or static_features.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/191M [00:00<?, ?B/s]

	Fine-tuning on the CPU detected. We recommend using a GPU for faster fine-tuning of Chronos.
	Saving fine-tuned model to /content/AutogluonModels/models/ChronosFineTuned[bolt_small]/W0/fine-tuned-ckpt
	Time limit exceeded... Skipping ChronosFineTuned[bolt_small].
Training timeseries model TemporalFusionTransformer. Training for up to 37.1s of the 185.4s of remaining time.
	-0.6479       = Validation score (-WAPE)
	37.63   s     = Training runtime
	7.32    s     = Validation (prediction) runtime
Training timeseries model DeepAR. Training for up to 35.1s of the 140.4s of remaining time.
	Time limit exceeded... Skipping DeepAR.
Stopping training due to lack of time remaining. Time left: -277.7 seconds
Not fitting ensemble due to lack of time remaining. Time left: -277.7 seconds
Training complete. Models trained: ['SeasonalNaive', 'TemporalFusionTransformer']
Total runtime: 864.05 s
Best model: TemporalFusionTransformer
Best model score: -0.6479
data with frequency 'IRREG' has been resamp

TypeError: TimeSeriesPredictor.evaluate() got an unexpected keyword argument 'forecasts'

In [24]:

# ----------------------------
# STEP : Evaluate
# ----------------------------

metrics = predictor.evaluate(test_data)
print("Evaluation metrics:\n", metrics)

data with frequency 'IRREG' has been resampled to frequency 'D'.


ValueError: Cannot reserve last 31 time steps for evaluation in some time series in data. Please make sure that data includes both historical and future data, and thatall time series have length > prediction_length (at least 32)

In [ ]:
prediction_length = 31

predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    target="Amount",
    eval_metric="WAPE",  # or "MAPE", "RMSE"
    path="AutogluonModelsForecast/"
)

predictor.fit(
    ts_data,  # full data
    presets="best_quality",
    time_limit=1800  # adjust as needed
)

# ----------------------------
# STEP 4: Forecast 31 future days
# ----------------------------
forecast = predictor.predict(ts_data)

# Save forecast
forecast_df = forecast.to_data_frame().reset_index()
forecast_df.to_csv("/content/forecast_next_31_days.csv", index=False)

In [ ]:
END

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=48,
    path="autogluon-m4-hourly",
    target="target",
    eval_metric="MASE",
)

predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=600,
)


Here we used the `"medium_quality"` presets and limited the training time to 10 minutes (600 seconds).
The presets define which models AutoGluon will try to fit.
For `medium_quality` presets, these are
simple baselines (`Naive`, `SeasonalNaive`),
statistical models (`ETS`, `Theta`),
tree-based models based on LightGBM (`RecursiveTabular`, `DirectTabular`),
a deep learning model `TemporalFusionTransformer`,
and a weighted ensemble combining these.
Other available presets for `TimeSeriesPredictor` are `"fast_training"`, `"high_quality"` and `"best_quality"`.
Higher quality presets will usually produce more accurate forecasts but take longer to train.

Inside `fit()`, AutoGluon will train as many models as possible within the given time limit.
Trained models are then ranked based on their performance on an internal validation set.
By default, this validation set is constructed by holding out the last `prediction_length` timesteps of each time series in `train_data`.


## Generating forecasts with `TimeSeriesPredictor.predict`

We can now use the fitted `TimeSeriesPredictor` to forecast the future time series values.
By default, AutoGluon will make forecasts using the model that had the best score on the internal validation set.
The forecast always includes predictions for the next `prediction_length` timesteps, starting from the end of each time series in `train_data`.

In [ ]:
predictions = predictor.predict(train_data)
predictions.head()


AutoGluon produces a _probabilistic_ forecast: in addition to predicting the mean (expected value) of the time series in the future, models also provide the quantiles of the forecast distribution.
The quantile forecasts give us an idea about the range of possible outcomes.
For example, if the `"0.1"` quantile is equal to `500.0`, it means that the model predicts a 10% chance that the target value will be below `500.0`.

We will now visualize the forecast and the actually observed values for one of the time series in the dataset.
We plot the mean forecast, as well as the 10% and 90% quantiles to show the range of potential outcomes.

In [ ]:
import matplotlib.pyplot as plt

# TimeSeriesDataFrame can also be loaded directly from a file
test_data = TimeSeriesDataFrame.from_path("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly_subset/test.csv")

# Plot 4 randomly chosen time series and the respective forecasts
predictor.plot(test_data, predictions, quantile_levels=[0.1, 0.9], max_history_length=200, max_num_item_ids=4);


## Evaluating the performance of different models

We can view the performance of each model AutoGluon has trained via the `leaderboard()` method.
We provide the test data set to the leaderboard function to see how well our fitted models are doing on the unseen test data.
The leaderboard also includes the validation scores computed on the internal validation dataset.

Note the test data includes both the forecast horizon (last `prediction_length` values of each time series) as well as the historical data (all except the last `prediction_last` values).

In AutoGluon leaderboards, higher scores always correspond to better predictive performance.
Therefore our MASE scores are multiplied by `-1`, such that higher "negative MASE"s correspond to more accurate forecasts.

In [ ]:
# The test score is computed using the last
# prediction_length=48 timesteps of each time series in test_data
predictor.leaderboard(test_data)


## Summary
We used `autogluon.timeseries` to make probabilistic multi-step forecasts on the M4 Hourly dataset.
Check out [Forecasting Time Series - In Depth](forecasting-indepth.ipynb) to learn about the advanced capabilities of AutoGluon for time series forecasting.